In [89]:
import pandas as pd
import requests
import lxml.html as html
import os
import datetime
import requests
from bs4 import BeautifulSoup

In [90]:
#Teniendo la url para hacer el web scraping
urlModalidad='https://admision.unmsm.edu.pe/Website20241/'

def get_modalidad_url(urlModalidad):
    response = requests.get(urlModalidad, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        urls = [urlModalidad+a['href'][::] for a in soup.select('tbody tr td.text-center a')]
    return urls


In [91]:
urlAdmision=get_modalidad_url(urlModalidad)

In [92]:
def tabla_modalidad(urlModalidad):
    response = requests.get(urlModalidad, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        contents = [a.text.lower() for a in soup.select('tbody tr td.text-center a')]
        tabla =pd.DataFrame(contents,columns=['Modalidad']).reset_index()
        tabla.rename(columns={'index': 'id_modalidad'},inplace=True)
    return tabla


In [93]:
#creando la tabla de modalidades
tabla_modalidad=tabla_modalidad(urlModalidad)

In [94]:
#Exportando las modalidades de Ingreso 
tabla_modalidad.to_csv('./data/tabla_modalidad.csv',index=False,sep=',')

In [95]:
#creando la funcion para obtener la los links de cada carrera por modaldidad 
def get_carrera_url(urlAdmision):
    response = requests.get(urlAdmision, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        urls = [urlAdmision[:-7:]+a['href'][1::] for a in soup.select('tbody tr td.text-center a')]
        return urls

In [96]:
def tabla_carrera(urlAdmision):
    response = requests.get(urlAdmision, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        tabla = [[a['href'][4:-7:],a.text.upper()] for a in soup.select('tbody tr td.text-center a')]
        tabla=pd.DataFrame(tabla,columns=['IdCarrera','Carrera'])
        return tabla

In [97]:
tabla_carrera=tabla_carrera(urlAdmision[0])
tabla_carrera.IdCarrera=tabla_carrera.IdCarrera.astype(int)

In [98]:
#opteniendo nuestra tabla de carreras 
tabla_carrera

,IdCarrera,Carrera
0,22,DERECHO
1,23,CIENCIA POLÍTICA
2,31,LITERATURA
3,33,FILOSOFÍA
4,34,LINGUÍSTICA
...,...,...
63,193,INGENIERÍA DE TELECOMUNICACIONES
64,194,INGENIERÍA BIOMÉDICA
65,201,INGENIERÍA DE SISTEMAS
66,202,INGENIERÍA DE SOFTWARE


In [99]:
#exportando nuestra tabla de carreras 
tabla_carrera.to_csv('./data/tabla_carrera.csv',sep=',',index=False)

In [100]:
url=urlModalidad

urlsModalidad=get_modalidad_url(url)

In [101]:
tabla_modalidad

,id_modalidad,Modalidad
0,0,educación básica regular (ebr) y educación bás...


In [102]:
#en esta funcion se introduce una lista
#devuelve el dataframe de todas las carreraspor modalidad y con id modaldiad
def get_df(urls,num):
    resultados = pd.DataFrame()
    for url in urls:
        tables = pd.read_html(url, encoding='utf-8')
        table = pd.concat(tables)
        resultados = pd.concat([resultados, table], ignore_index=True)
    resultados['IdModalidad']=num
    return resultados
    

In [103]:
#recolectamos todas las url opr modalidad de ingreso
urlsModalidad=get_modalidad_url(url)
urlsCarrera=[]
for i , urlModalidad in enumerate(urlsModalidad):
    #guardando las url de carrera
    urlsCarrera.append(get_carrera_url(urlModalidad))


In [108]:
#creando un bucle para extraer la informacion de todos los postulantes ausentes y que ingresaron
todo = pd.DataFrame()
for i in range(len(urlsCarrera)):
    resultado=get_df(urlsCarrera[i],i)
    todo = pd.concat([todo, resultado], ignore_index=True)

In [109]:
#exportamdo los resultados de examen de admision 
#hacer un for donde extraigamos toda la data y le incluya una columna para especificar su modalidad
todo.to_csv('./data/tabla_resultados.csv',index=False,sep=',')

In [110]:
todo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CODIGO                     18484 non-null  int64  
 1   APELLIDOS Y NOMBRES        18484 non-null  object 
 2   ESCUELA PROFESIONAL        18484 non-null  object 
 3   PUNTAJE FINAL              18278 non-null  float64
 4   MERITO E.P                 1727 non-null   float64
 5   OBSERVACI&OacuteN          1933 non-null   object 
 6   DETALLE OBSERVACI&OacuteN  249 non-null    object 
 7   IdModalidad                18484 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [86]:
todo.head()

,CODIGO,APELLIDOS Y NOMBRES,ESCUELA PROFESIONAL,PUNTAJE FINAL,MERITO E.P,OBSERVACI&OacuteN,IdModalidad
0,855168,"ABAD GRANDA, ANDRE FAHET",MEDICINA HUMANA,774.75,NaN,NaN,0
1,860373,"ABAD NEYRA, ANDREA TAIS",MEDICINA HUMANA,299.0,NaN,NaN,0
2,894207,"ABAD ONCOY, MARIA BELÉN",MEDICINA HUMANA,441.75,NaN,NaN,0
3,875677,"ABAL PRINCIPE, INGRID",MEDICINA HUMANA,530.5,NaN,NaN,0
4,855333,"ABANTO ARAUJO, PERCY ANGEL",MEDICINA HUMANA,584.625,NaN,NaN,0
